In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from arcgis.geometry import Geometry, SpatialReference, Point, Polygon
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
import matplotlib.pyplot as plot
from matplotlib.pyplot import figure
import matplotlib.image as img

# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
gis = GIS("https://www.arcgis.com", "ezhatcher", getpass())

········


In [2]:
import plotly
import plotly.plotly as py
import plotly.offline as py_off
import plotly.tools as tls
plotly.__version__

'3.1.1'

In [3]:
#Define the function to access a field within a layer
def access_lyr(content,out_fld):
    base = content.layers
    lyr = base[0]
    topic_data = lyr.query(where="CommunityName = '" + str(community) + "'",out_fields=out_fld)
    return topic_data.df

In [4]:
import uuid,json,arcpy

def getExtent(fromFeature, sp_ext_ref=102100):
    g = Geometry(fromFeature.geometry)
    if g.type.lower() == 'point':
        bg = Point({"x": g.x, "y": g.y, "spatialReference": {"wkid": sp_ext_ref}}).project_as(102100)
        gbuf = bg.buffer(2800) # assume meters, 1609.344 ~ 1 mile
        return gbuf.extent
    
    return g.extent

def getMapAction(fromFeature, forWebmap, sp_ref=102100, lyrNdx=0, label_field="OBJECTID", label_prefix="Feature Extent", add_popup=False):
    # TODO - make sure input parameters are valid, since we are just inserting them into the mapaction schema
    
    # pull out a value from the feature to display in our anchor tag
    lbl_val = fromFeature.attributes[fromFeature.fields[0]]
    if label_field in fromFeature.fields:
        lbl_val = fromFeature.attributes[label_field]
    
    # format the label
    lbl_frmtd = '{0}{1}'.format(label_prefix, lbl_val)
    
    # get the geometry extent
    # TODO - Need to validate this or default to a valid extent
    ext = getExtent(fromFeature, sp_ext_ref=sp_ref)
    
    # find the oid field
    oid_field = [f for f in fromFeature.fields if 'objectid' in str(f.lower())][0]
    if len(oid_field) <= 0 or oid_field is None:
        print('objectid not in field list, disabling popup')
        add_popup = False
        
    wm = WebMap(forWebmap)
    layer = wm.layers[lyrNdx]
    
    lyr_lst = []
    main_lyr = wm.layers[lyrNdx].id
    main_lyr
    for lyr in wm.layers:
        if lyr.id != main_lyr:
            lyr_lst.append(
            {
                "id": lyr.id,
                "visibility":'false'
            })
    
    #spatialRef = FeatureLayer(layer.url).properties.sourceSpatialReference.wkid ## Updated 9/4 EZH
    if add_popup and layer is None:
        print('layer not found, disabling popup')
        add_popup = False
    
    point_ft = Geometry(fromFeature.geometry)
    project = Point({"x":point_ft.x, "y": point_ft.y, "spatialReference": {"wkid":sp_ref}}).project_as(102100)
    
    popup_def = None
    if add_popup:
        popup_def = {
            "layerId": layer.id,
            "fieldName": oid_field, # we should probably get this dynamically and not hard code
            "fieldValue": fromFeature.attributes[oid_field], # same with this one
            "anchorPoint": {
                "x": project.x,
                "y": project.y,
                "spatialReference": {
                    "wkid": sp_ref
                }
            }
        }
    
    # create a unique id
    # ts = int(round(time.time() * 1000)) this isn't precise enough, gets duplicated in loop
    ts = str(uuid.uuid4())
    #print('id : {0}'.format(ts))
    
    # insert values into our map action schema
    ma = {
        "id":"MJ-ACTION-{0}".format(ts),
        "type": "media",
        "media": {
            "type":"webmap",
            "webmap": {
                "id":forWebmap.id,
                "extent": {
                    "xmin": ext[0],
                    "ymin": ext[1],
                    "xmax": ext[2],
                    "ymax": ext[3],
                    "spatialReference":{
                        "wkid": sp_ref ## Updated 9/4 EZH
                    }
                },
                "layers": lyr_lst,
                "popup": popup_def,
                "overview":{
                    "enable": False,
                    "openByDefault": False
                },
                "legend": {
                    "enable": False,
                    "openByDefault": False
                },
                "geocoder":{
                    "enable": False
                },
                "altText": ""
            }
        }
    }
    
    # create the html string
    content_anchor = '<a data-storymaps="{0}" data-storymaps-type="media">{1}</a>'.format(ma['id'], lbl_frmtd)
    
    # hand back our results to whoever called this function
    return {"content_link": content_anchor, "map_action": ma}

In [5]:
#Define the base template storymap using the item ID -- Hoonah Alaska
seedappItem = gis.content.get("2a51cb9cc9d54b57a52c67d8efdb5eba") #"2a51cb9cc9d54b57a52c67d8efdb5eba"
#Clone the storymap
contentManager = arcgis.gis.ContentManager(gis)
clones = contentManager.clone_items(items=[seedappItem],search_existing_items=False)
clonedStorymap = storymap.JournalStoryMap(clones[0])

In [6]:
#Use the base feature class that has the listing of communities for which a storymap will be created-- currently, it's the 
#boundaries feature service, which also defines the map extent as the user scrolls through the panels
community_base = gis.content.get("8499c3bc4ff141588f5c9ab1db99b0fb")
find_comm = community_base.layers[0].query(where="NAME IS NOT NULL",out_fields="NAME").df.NAME
community_lst = []
for c in find_comm:
    community_lst.append(c)

In [7]:
#Save the storymap clone using the new community name and information
community = 'Nome'
clonedStorymap.save(title=str(community) +" Storymap Test", description="Test cloning script result", tags="Test,Storymap")
sections = clonedStorymap.properties["values"]["story"]["sections"]

In [8]:
#Access the URL for the main photo from the Photo Link feature class
photo_base = gis.content.get("6d9d437bbe2d48bfa87b4d9fca41f80d")
find_photo = access_lyr(photo_base,'PhotoLink')
main_pic = find_photo.PhotoLink[0]
if str(main_pic) == 'None':
    print('There is no picture for ' + str(community))
else:
    print('Picture found for ' + str(community) + ' at ' + str(main_pic))

Picture found for Nome at https://dcced.maps.arcgis.com/sharing/rest/content/items/3bc15a7b9344496dacfc27f492e4685e/data


In [9]:
#Identify/define the storymap panels
panel_toc = sections[0]
panel_ch = sections[1]
panel_demog = sections[2]
panel_transport = sections[3]
panel_edu = sections[4]
panel_econ = sections[5]
panel_util = sections[6]
panel_ANCSA = sections[7]
panel_contacts = sections[8]

panel_list = []
panel_list.append(panel_demog)
panel_list.append(panel_transport)
panel_list.append(panel_edu)
panel_list.append(panel_econ)
#panel_list.append(panel_util) #problem with bounding box...
#panel_list.append(panel_ANCSA) #problem with bounding box... 
panel_list.append(panel_contacts)


#Modify the main title and photo in the main Table of Contents panel
panel_toc["title"] = '<span style="font-size:40px">' + str(community) + ', Alaska</span>'
if str(main_pic) == 'None':
    panel_toc["media"]["image"]["url"] = "http://dcced.maps.arcgis.com/sharing/rest/content/items/34bf2ca84faa4b258712ed7a5422d0d5/data"
else:
    panel_toc["media"]["image"]["url"] = main_pic
clonedStorymap.save()

True

In [10]:
pop_base = gis.content.get("b1c57544c64849f6b626332bd05785cb")
current_pop = pop_base.layers[0].query(where="(CommunityName = '" + str(community) + "') AND DataYear = 2017", out_fields = "*").df.Population[0]
incrp_type = access_lyr(photo_base,'CommunityTypeName').CommunityTypeName[0]
incrp_date = community_base.layers[0].query(where = "NAME = '" + str(community) + "'", out_fields = 'INCRP_DATE').df.INCRP_DATE[0]
print("Population: " + str(current_pop) + " Inc. type: " + str(incrp_type), incrp_date)

panel_toc["content"] = '<style type="text/css">.blue {\n     display: inline-block;\n     background-color: #2c7bb6;\n     border-color: #2c7bb6 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><span style="font-weight: bold;">Current Population: ' + str(current_pop) + '</span></p><p>&nbsp;</p><p>Incorporation Date: ' + str(incrp_date) + '</p><p>Incorporation Type: ' + str(incrp_type) + '</p><p>&nbsp;</p>\n<p><span style="font-size:28px">Table of Contents</span><a data-storymaps="MJ-ACTION-1524782969520" data-storymaps-type="navigate"> </a></p>\n\n<p>&nbsp;</p>\n\n<p><a data-storymaps="MJ-ACTION-1524782969520" data-storymaps-type="navigate">Culture and History</a></p>\n\n<p><span style="font-size:20px"><a class="peach" data-storymaps="MJ-ACTION-1524783369112" data-storymaps-type="navigate">Demographics</a></span></p>\n\n<p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1524784937326" data-storymaps-type="navigate">Transportation</a></span></p>\n\n<p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1524786044096" data-storymaps-type="navigate">Education</a></span></p>\n\n<p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532548226455" data-storymaps-type="navigate">Economy</a></span></p>\n\n<p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532636740461" data-storymaps-type="navigate">Utilities</a></span></p>\n\n<p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532636735244" data-storymaps-type="navigate">ANCSA&nbsp;</a></span></p>\n\n<p><span style="font-size:20px"><a data-storymaps="MJ-ACTION-1532636729718" data-storymaps-type="navigate">Community Contacts</a></span></p>\n\n<p>&nbsp;</p>\n\n<p>&nbsp;</p>\n\n<p>&nbsp;</p>\n'
clonedStorymap.save()

Population: 3691 Inc. type: 1st Class City 1901


True

In [11]:
#Access the webmap and clone the webmap/update map filters
current_webmap = gis.content.get("bdbb441b8b5d4ef2bc1a519c44e49c6b")
wm = arcgis.mapping.WebMap(webmapitem=current_webmap)
wm_properties = {'title':'OpenData_Storymap_' + str(community),'snippet':'Test webmap clone','tags':['dcra','test']}
wm.save(wm_properties,folder='Storymap Webmaps') 
#Find the new webmap and rename it
new_map_search = gis.content.search('OpenData_Storymap_' + str(community), item_type="Web Map")
webmap = new_map_search[0]
new_webmap = webmap.id
#Define the webmap item to access and filter layers
upd_webmap = gis.content.get(new_webmap)
wm_instory = arcgis.mapping.WebMap(webmapitem=upd_webmap)


In [13]:
#UPDATE WEBMAP LAYER FILTERS

#Find ANCSA Regional Corporation
region_lyr = gis.content.get("5522cf5f5e5d4d52abe6e7df47d5d241").layers[0].query(where="Community_Name = '" + str(community) + "'", out_fields = "*")
ANCSA_rgn = region_lyr.df.ANCSA_Regional_Corporation[0]
econ_rgn = region_lyr.df.Economic_Region[0]
sch_distr = region_lyr.df.School_District[0]
#Taxes
wm_instory.layers[0]["layerDefinition"]["definitionExpression"] = "(CommunityName = '" + str(community) +"') AND (RevenueYear = 2016)"

#Income and Poverty
wm_instory.layers[1]["layerDefinition"]["definitionExpression"] = "CommunityName = '" + str(community) + "'"

#Municipality Contacts
wm_instory.layers[2]["layerDefinition"]["definitionExpression"] = "CommunityName = '" + str(community) + "'"

#School Enrollment
wm_instory.layers[3]["layerDefinition"]["definitionExpression"] = "CommunityName = '" + str(community) + "'"

#DCCED Certified Population
wm_instory.layers[4]["layerDefinition"]["definitionExpression"] = "CommunityName = '" + str(community) + "'"

#Ferries
wm_instory.layers[5]["layerDefinition"]["definitionExpression"] = "CommunityName = '" + str(community) + "'"

#Healthcare facilities
wm_instory.layers[6]["layerDefinition"]["definitionExpression"] = "CommunityName = '" + str(community) + "'"

#Community Regions Overview
wm_instory.layers[7]["layerDefinition"]["definitionExpression"] = "CommunityName = '" + str(community) + "'"

#Alaska Road System
#wm_instory.layers[8]

#Boro_REAAs
#wm_instory.layers[9]

#Alaska Airports
wm_instory.layers[10]["layerDefinition"]["definitionExpression"] = "CommunityName = '" + str(community) + "'"

#City Bounding Area
wm_instory.layers[11]["layerDefinition"]["definitionExpression"] = "NAME = '" + str(community) + "'"

#ANCSA Regional Corporation
wm_instory.layers[12]["layerDefinition"]["definitionExpression"] = "NAT_CORP = '" + str(ANCSA_rgn) + "'"

#Economic Region Employment and Wages
wm_instory.layers[13]["layerDefinition"]["definitionExpression"] = "CommunityName = '" + str(econ_rgn) + "'"

#School Disricts
wm_instory.layers[14]["layers"][0]["layerDefinition"]["definitionExpression"] = "School_District = '" + str(sch_distr) + "'"

wm_instory.update()
print(str(community) + " webmap id: " + str(new_webmap))

Nome webmap id: 005c8a18aa3c4509b70bf08b3713bc6a


In [14]:
### STORYMAP PANELS WITH CITY EXTENT ###
#Set up zoom scale (x max and min, y max and min) for map actions 
bounds_ft = gis.content.get("8499c3bc4ff141588f5c9ab1db99b0fb")
ft_lyr = FeatureLayer(bounds_ft)
boundary_lyr = ft_lyr.url.layers[0].query(where="NAME LIKE '%" + str(community) + "%'", out_fields="*").features[0].geometry
bounds_ext = Polygon(boundary_lyr).extent
c_ext = Geometry({
    'xmin':bounds_ext[0],
    'ymin':bounds_ext[1],
    'xmax':bounds_ext[2],
    'ymax':bounds_ext[3],
    'spatialReference':{
        'wkid':102006
    }
}).project_as(3857)

print(str(community) + " extent: " + str(c_ext))

#Update primary zoom scale of each panel based on the minimum bounding envelope feature service
# item id = "8499c3bc4ff141588f5c9ab1db99b0fb" using previously defined zoom scale
for p in panel_list:
    p["media"]["webmap"]["id"] = new_webmap
    ext1 = p["media"]["webmap"]["extent"]
    ext1["xmin"] = c_ext.xmin
    ext1["xmax"] = c_ext.xmax
    ext1["ymin"] = c_ext.ymin
    ext1["ymax"] = c_ext.ymax
    print(str(p["title"]) + " main extent updated.")
    clonedStorymap.save()


Nome extent: {'xmin': -18427977.885244764, 'ymin': 9465769.721408887, 'xmax': -18402327.418988876, 'ymax': 9495095.129160557, 'spatialReference': {'wkid': 102100, 'latestWkid': 3857}}
<span style="font-size:26px">Demographics</span> main extent updated.
<span style="font-size:26px">Transportation</span> main extent updated.
<span style="font-size:26px">Education</span> main extent updated.
<span style="font-size:26px">Economy</span> main extent updated.
<span style="font-size:26px">Contacts</span> main extent updated.


In [15]:
### CULTURE HISTORY PANEL ###

#Access the culture and history from the culture/history feature class (IF THE PANEL ORDER CHANGES, MAKE SURE INDEX STILL APPLIES)
panel_ch["media"]["webmap"]["id"] = new_webmap
ch_ft = gis.content.get("3f99cf21a0da42c4bf8b15632e958df9")
culture_lookup = access_lyr(ch_ft,'CommunityCulture')
culture = culture_lookup.CommunityCulture[0]
history_lookup = access_lyr(ch_ft,'CommunityHistory')
history = history_lookup.CommunityHistory[0]

#Populate the panel with the new culture history info
panel_ch["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;<p>'+ history + '&nbsp;</p>\n\n<p>\n\n<p>&nbsp;<p>'+ culture + '&nbsp;</p>\n'
panel_ch["media"]["webmap"]["altText"] = 'Map showing the location of ' + str(community) + ', Alaska in the context of the state and borough/REAA boundaries.'
clonedStorymap.save()

True

In [16]:
### DEMOGRAPHICS PANEL ###

#Update the zoom extent for the demographics panel map action (tied to 'Historic Census' button)
panel_demog["contentActions"][1]["media"]["webmap"]["id"] = new_webmap
demog_ext = panel_demog["contentActions"][1]["media"]["webmap"]["extent"]
demog_ext["xmin"] = c_ext.xmin
demog_ext["xmax"] = c_ext.xmax
demog_ext["ymin"] = c_ext.ymin
demog_ext["ymax"] = c_ext.ymax

#Update the main DCCED popup
#demog_popup = panel_demog["contentActions"][1]["media"]["webmap"]["popup"]

panel_demog["media"]["webmap"]["altText"] = 'Map showing aerial imagery of ' + str(community) + ', Alaska and a pop-up box displaying current and historic DCCED certified population counts from 2011 to present.'


### DEMOGRAPHICS PANEL ###
#Update popup extents
dcced_pop = FeatureLayer('https://maps.commerce.alaska.gov/server/rest/services/Demographics/Population_DCCED_View/MapServer/0')
pop_lyr = dcced_pop.query(where="CommunityName = '" + str(community) + "'", out_fields = "*")
point_ft = Geometry(pop_lyr.features[0].geometry)
project_dcced = Point({"x":point_ft.x, "y": point_ft.y, "spatialReference": {"wkid":4326}}).project_as(102100)
panel_demog["media"]["webmap"]["popup"]["fieldName"] = 'OBJECTID'
panel_demog["media"]["webmap"]["popup"]["fieldValue"] = int(pop_lyr.df.OBJECTID[0])
panel_demog["media"]["webmap"]["popup"]["anchorPoint"]["x"] = project_dcced.x
panel_demog["media"]["webmap"]["popup"]["anchorPoint"]["y"] = project_dcced.y

print("Demographics popup updated.")

clonedStorymap.save()

Demographics popup updated.


True

In [17]:
### DEMOGRAPHICS PANEL ###

#Embed census plot from plotly
py.sign_in('ezhatcher','vCIBrBR5RpQhanTzQBRZ')

#Historical census plot for demographics panel
census_base = gis.content.get("b475ec701f25448aa6e03cb16d94b6f0") #Census locations all locations
pop_df = access_lyr(census_base,'CensusPopulation, CensusYear')

graph = plotly.graph_objs.Scatter(
    x = pop_df.CensusYear,
    y = pop_df.CensusPopulation
)

layout = plotly.graph_objs.Layout(
    title=str(community) + ' Historical Census Population Counts',
    )

data = [graph]
fig = plotly.graph_objs.Figure(data=data, layout=layout)
config={'showLink': False}
plot_url = py.plot(fig, filename= str(community) + '-census',config=config)
embed_plt = tls.get_embed(plot_url)

panel_demog["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783883308" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p>Every June the Commissioner of the Department of Commerce Community Economic Development certifies community population figures. The most recent figures are displayed here.&nbsp;</p>\n\n<p>&nbsp;</p>\n\n<p> Historic census population counts for the community also provide a century\'s worth of change in this community.&nbsp;</p>\n\n<div>' + str(embed_plt) +'</div>\n\n<p>More information about demographics can be found <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=577407acfbc6433389006d099cb25971" target="_blank">here.&nbsp;</a></p>\n'
clonedStorymap.save()
print("Historic census plot posted to panel")

Historic census plot posted to panel


In [18]:
###TRANSPORTATION PANEL###
#Update panel text using transportation overview feature class (item id = "2076ce8b6744430880e1c8ab923cd720")
find_text = gis.content.get("4b1dc0106a7c45dab6bfcdcef18bb84d")
access_text = find_text.layers[0].query(where="CommunityName = '" + str(community) + "'",out_fields='Description').df

#Create airport listing table in panel
arpt_base = gis.content.get("40601fd695c74a6d8a53303ece73ecd1")
arpt_data = access_lyr(arpt_base,'EntityName')
if str(arpt_data) == 'Empty DataFrame\nColumns: []\nIndex: []':
    print("There are no public use airports in " + str(community))
    panel_transport["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n}\n</style>\n<style type="text/css">.btn-orange {\n    display: inline-block;\n    background-color: #f0ad4e;\n    border-color: #f0ad4e !important;\n    color: #fff !important;\n    padding: 0px 2px;\n    border-radius: 12px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524784108786" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p>' + access_text.Description[0] + '</p>\n\n<p>&nbsp;</p>\n\n<p>&nbsp;</p>\n\n<p>&nbsp;</p>\n\n<p>More detailed information about Transportation in Alaska can be found <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=3de3ebccbb6b4ba8a9c64609e96199ba" target="_blank"><strong>here</strong></a>.&nbsp;</p>\n'
else:
    def airport_lst(airports):
        arpt_listing = ["<table border = '1'><tr><th width='70%'>Airport</th><th width='30%'>Airport Code</th></tr><tbody>"]
        for a in airports.EntityName:
            airport_data = arpt_base.layers[0].query(where="EntityName = '" + str(a) + "'",out_fields='EntityName, AirportCode')
            airport = airport_data.df.EntityName[0]
            arpt_code = airport_data.df.AirportCode[0]
            ma = getMapAction(airport_data.features[0], webmap, sp_ref=102006, lyrNdx=10, label_field='EntityName',label_prefix="", add_popup=False)
            panel_transport["contentActions"].append(ma["map_action"])
            arpt_listing.append("<tr><td>" + ma['content_link'] + "</td><td>" + str(arpt_code) + "</td><td>")
        arpt_listing.append("</tbody></table>")
        return(' '.join(arpt_listing))
    airport_lst(arpt_data)
    
    panel_transport["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n}\n</style>\n<style type="text/css">.btn-orange {\n    display: inline-block;\n    background-color: #f0ad4e;\n    border-color: #f0ad4e !important;\n    color: #fff !important;\n    padding: 0px 2px;\n    border-radius: 12px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524784108786" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p>' + str(access_text.Description[0]) + '</p>\n\n<p>&nbsp;</p><p>Click on the icons in the map to see specific information about each airport or ferry terminal.</p>\n\n<p>&nbsp;</p>\n\n<p>' + str(airport_lst(arpt_data)) + '</p><p>&nbsp;</p>\n\n<p>More detailed information about Transportation in Alaska can be found <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=3de3ebccbb6b4ba8a9c64609e96199ba" target="_blank"><strong>here</strong></a>.&nbsp;</p>\n'
    clonedStorymap.save()
    print("Transportation table posted to panel.")

Transportation table posted to panel.


In [19]:
### ECONOMY PANEL ###

#Employment rate from employment and wages data
emp_wgs_base = gis.content.get("601a1796d41d485992a18e2974845532").layers[0]
emp_wgs_data = emp_wgs_base.query(where="(CommunityName = '" + str(community) + " city') AND DataYear = 2016", out_fields="*").df
emp_rate = round(float(emp_wgs_data.ResidentsEmployed[0]/emp_wgs_data.ResidentsAge16AndOver * 100),1)
emp_rate

#Taxes data
tax_base = gis.content.get("70a364eae3514531a280e4696b54c6e9").layers[0]
tax_data = tax_base.query(where="(CommunityName = '" + str(community) +"') AND RevenueYear = 2016", out_fields = "*")
sales_tax = tax_data.df.SalesTaxPercentage[0]
prop_tax = tax_data.df.PropertyTaxMills[0]
other_tax = tax_data.df.OtherTaxes[0]
prop_tax_rev = tax_data.df.PropertyTaxRevenue[0]
sales_tax_rev = tax_data.df.SalesTaxRevenue[0]
total_tax_rev = tax_data.df.TotalTaxRevenue[0]
tax_table = ['<table><tr><th width="70%"></th><th width="30%"></th></tr>',
             '<tr><td>Property Tax Mills:</td><td>' + str(prop_tax) + '</td></tr>',
            '<tr><td>Sales Tax Rate: </td><td>' + str(sales_tax) + '</td></tr>',
            '<tr><td>Other Taxes: </td><td>' + str(sales_tax) + '</td></tr>',
            '<tr><td>Total Tax Revenue: </td><td> $' + str(total_tax_rev) + '</td></tr>',
            '<tr><td>Sales Tax Revenue: </td><td> $' + str(sales_tax_rev) + '</td></tr>']
embed_tax_tbl = ' '.join(tax_table)

#Income and poverty data
#inc_pov_base = gis.content.get("")

panel_econ["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p><b><u>Employment Rate: </u></b>' + str(emp_rate) + '%</p><p>&nbsp;</p>\n\n<div><p><b><u>Taxes: </u></b> ' + str(embed_tax_tbl) + '</p></div><p>&nbsp;</p><p><b><u>Income and Poverty:</u></b></p>\n\n<p>More information can be found in the <a href="http://DCCED.maps.arcgis.com/apps/MapJournal/index.html?appid=bb631449256346db81bc026339f0e60c" target="_blank">Alaska Taxable.</a></p>\n'

clonedStorymap.save()
print("Economic data posted to economy panel.")

Economic data posted to economy panel.


In [20]:
###EDUCATION PANEL###

#CUSTOM POPUP
ak_schools = gis.content.get("1f8928a4bae646ef884be3b6a0b8bae8")
#If using layer published to portal...
community_sch_lst = access_lyr(ak_schools,'EntityName,PhysicalAddress,PhysicalCity,PhysicalZip')
#If using layer published to cloud...
#community_sch_lst = ak_schools.layers[0].query(where="CommunityN = '" + str(community) + "'", out_fields = 'EntityName,PhysicalAd,PhysicalCi,PhysicalZi').df
output = str(ak_schools.layers[0].query(where="CommunityName = '" + str(community) + "'",out_fields = "EntityName").df)
if output == 'Empty DataFrame\nColumns: []\nIndex: []':
    print("There are no schools in " + str(community))
    panel_edu["content"] = '<p>&nbsp;</p>\n\n<p>\n<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n</style>\n</p>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524785467480" data-storymaps-type="navigate">Back to Table of Contents</a><br>\n<style type="text/css">\n</style>\n</p>\n\n<p>&nbsp;</p>\n\n<p>There are no schools open in' + str(community) + '</p><p>&nbsp;</p>\n\n<p>For more information about Alaska schools and education, click <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=cff461c2397d4c66ae2c28dbfb62319f" target="_blank">here</a>.</p>\n'
else:
    print("Creating " + str(community) + " schools table...")
    schools = community_sch_lst.EntityName
    def school_lst(entities):
        school_list = ["<table border = '1'><tr><th width='45%'>School</th><th width='45%'>Address</th><th width='10%'>Zip Code</th></tr><tbody>"]
        for e in entities:
            #print(e)
            #school_data = ak_schools.layers[0].query(where="EntityName = '" + str(e) + "'",out_fields= 'EntityName,PhysicalAd,PhysicalCi,PhysicalZi')
            school_data = ak_schools.layers[0].query(where="EntityName = '" + str(e) + "'",out_fields='EntityName,PhysicalAddress,PhysicalCity,PhysicalZip')
            address = school_data.df.PhysicalAddress[0]
            city = school_data.df.PhysicalCity[0]
            zipcode = school_data.df.PhysicalZip[0]
            ma = getMapAction(school_data.features[0], webmap, lyrNdx=4, label_field="EntityName", label_prefix="", add_popup=True)
            #print(ma)
            #panel_edu["content"] += ma['content_link']
            panel_edu["contentActions"].append(ma["map_action"])
            school_list.append("<tr><td>" + ma['content_link'] + "</td><td>" + address + "</td><td>" + zipcode + '</td></tr>')
        school_list.append("</tbody></table>")
        return(' '.join(school_list))
    
    school_lst(schools)
    ###EDUCATION PANEL###
    #Update panel schools listing
    school_dist = access_lyr(ak_schools,'SchoolDistrict').SchoolDistrict[0]
    panel_edu["content"] =  '<p>&nbsp;</p>\n\n<p>\n<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n</style>\n</p>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524785467480" data-storymaps-type="navigate">Back to Table of Contents</a><br>\n<style type="text/css">\n</style>\n</p>\n\n<p>&nbsp;</p>\n\n<p>' + str(community) + " schools are part of " + str(school_dist) + ". </p>\n\n<p>&nbsp;</p><p>" + str(school_lst(schools)) + '</p><p>&nbsp;</p>\n\n<p>For more information about Alaska schools and education, click <a href="http://DCCED.maps.arcgis.com/apps/webappviewer/index.html?id=cff461c2397d4c66ae2c28dbfb62319f" target="_blank">here</a>.</p>\n'
    panel_edu["content"]
    clonedStorymap.save()
    print("School table posted to the education panel.")
    

Creating Nome schools table...
School table posted to the education panel.


In [21]:
### ANCSA PANEL ###

extMain = panel_ANCSA["media"]["webmap"]["extent"]
extMain["xmin"] = c_ext.xmin
extMain["xmax"] = c_ext.xmax
extMain["ymin"] = c_ext.ymin
extMain["ymax"] = c_ext.ymax

def getPolyExtent(fromFeature):
    g = Polygon(fromFeature.geometry).extent
    bg = Geometry({"xmin": g[0], "ymin": g[1], "xmax": g[2], "ymax": g[3], "spatialReference": {"wkid": 102006}})
    return bg

def getPolyMapAction(fromFeature, forWebmap, lyrNdx=0, label_field="OBJECTID", label_prefix="Feature Extent", add_popup=False):
    # TODO - make sure input parameters are valid, since we are just inserting them into the mapaction schema
    
    # pull out a value from the feature to display in our anchor tag
    lbl_val = fromFeature.attributes[fromFeature.fields[0]]
    if label_field in fromFeature.fields:
        lbl_val = fromFeature.attributes[label_field]
    
    # format the label
    lbl_frmtd = '{0}{1}'.format(label_prefix, lbl_val)
    
    # get the geometry extent
    # TODO - Need to validate this or default to a valid extent
    ext = getPolyExtent(fromFeature)
    
    # find the oid field
    oid_field = [f for f in fromFeature.fields if f.lower() == 'objectid'][0]
    if len(oid_field) <= 0 or oid_field is None:
        print('objectid not in field list, disabling popup')
        add_popup = False
    
    wm = WebMap(forWebmap)
    layer = wm.layers[lyrNdx]
    #spatialRef = FeatureLayer(layer.url).properties.sourceSpatialReference.wkid ## Updated 9/4 EZH
    if add_popup and layer is None:
        print('layer not found, disabling popup')
        add_popup = False
    
    popup_def = None
    if add_popup:
        popup_def = {
            "layerId": layer.id,
            "fieldName": oid_field, # we should probably get this dynamically and not hard code
            "fieldValue": fromFeature.attributes[oid_field], # same with this one
            "anchorPoint": {
                "x": Geometry(fromFeature.geometry).centroid[0],
                "y": Geometry(fromFeature.geometry).centroid[1],
                "spatialReference": {
                    "wkid": 102006
                }
            }
        }
    
    # create a unique id
    # ts = int(round(time.time() * 1000)) this isn't precise enough, gets duplicated in loop
    ts = str(uuid.uuid4())
    #print('id : {0}'.format(ts))
    
    # insert values into our map action schema
    ma = {
        "id":"MJ-ACTION-{0}".format(ts),
        "type": "media",
        "media": {
            "type":"webmap",
            "webmap": {
                "id":forWebmap.id,
                "extent": {
                    "xmin": ext.xmin,
                    "ymin": ext.ymin,
                    "xmax": ext.xmax,
                    "ymax": ext.ymax,
                    "spatialReference":{
                        "wkid": 102006 ## Updated 9/4 EZH
                    }
                },
                "layers": None,
                "popup": popup_def,
                "overview":{
                    "enable": False,
                    "openByDefault": False
                },
                "legend": {
                    "enable": False,
                    "openByDefault": False
                },
                "geocoder":{
                    "enable": False
                },
                "altText": ""
            }
        }
    }
    
    # create the html string
    content_anchor = '<a data-storymaps="{0}" data-storymaps-type="media">{1}</a>'.format(ma['id'], lbl_frmtd)
    
    # hand back our results to whoever called this function
    return {"content_link": content_anchor, "map_action": ma}

In [22]:
### ANCSA PANEL ###

plats_lyr = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/ANCSA/ANCSA_14c_Plats/MapServer/0')
ANCSA_fts = plats_lyr.query(where="COMMUNITY_NAME = '" + str(community) + "'", out_fields="*")
if str(ANCSA_fts.features) == '[]':
    print("This is community is not included in the ANCSA 14(c)(3) plat inventory.")
    panel_ANCSA["media"]["webmap"]["popup"] = 'null'
else:
    point_ft = Geometry(ANCSA_fts.features[0].geometry)
    project = Point({"x":point_ft.x, "y": point_ft.y, "spatialReference": {"wkid":102006}}).project_as(102100)
    ANCSA_oid = ANCSA_fts.df.OBJECTID[0]
    panel_ANCSA["media"]["webmap"]["popup"]["fieldValue"] = int(ANCSA_oid)
    print("ANCSA popup object ID updated to object ID " + str(ANCSA_oid))
    panel_ANCSA["media"]["webmap"]["popup"]["anchorPoint"]["x"] = project.x
    panel_ANCSA["media"]["webmap"]["popup"]["anchorPoint"]["y"] = project.y

clonedStorymap.save()


ANCSA popup object ID updated to object ID 274


True

In [23]:
### ANCSA PANEL ###
apprx_rgn = ANCSA_rgn[:9]
ANCSA_corp = FeatureLayer(wm_instory.layers[12].url).query(where="NAT_CORP LIKE '%" + str(apprx_rgn) + "%'", out_fields="*")
ANCSA = ANCSA_corp.features[0]

ANCSA_ma = getPolyMapAction(ANCSA, webmap, lyrNdx=12, label_field="NAT_CORP", label_prefix=" ", add_popup=False)
panel_ANCSA["contentActions"].append(ANCSA_ma["map_action"])
panel_ANCSA["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p><p>&nbsp;</p><div><p> ANCSA Regional Corporation: ' + ANCSA_ma['content_link'] + '</p></div><p>&nbsp;</p><p>More information about ANCSA 14(c)3 Land Conveyances can be found <a href="http://dcced.maps.arcgis.com/apps/webappviewer/index.html?id=24427049328b40cdab001cae38883d7a" target="_blank">here.</a>&nbsp;</p>\n'
clonedStorymap.save()
print("ANCSA map action added to panel.")

ANCSA map action added to panel.


In [24]:
###CONTACTS PANEL###
#Find Municipality Contact Information
contacts_lyr = gis.content.get("e8b073f033b74bebbaa365e6dc6b6713")
contacts_lyr
contact_info = contacts_lyr.layers[0].query(where="City = '" + str(community) + "'",out_fields='EntityName,Address,City,Zip,Phone,Fax,Website,Email')
contact_info.df


,Address,City,Email,EntityName,Fax,OBJECTID,Phone,Website,Zip,SHAPE
0,PO Box 281,Nome,bhammond@nomealaska.org,City of Nome,9074435345,94,9074436663,http://www.nomealaska.org/,99762,"{'x': -18413196.6569, 'y': 9477390.954800002, ..."


In [25]:
###CONTACTS PANEL###
#Write municipality info to table
output = str(contact_info.df)
if output == 'Empty DataFrame\nColumns: []\nIndex: []':
    print(str(community) + ' is not a municipality. No municipal contact info will be written to panel.')
    panel_contacts["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<p>To find all community contacts and to download mailing labels visit the <a href="http://dcced.maps.arcgis.com/apps/webappviewer/index.html?id=148ae1328f9549acb46cfa80366ed37e/data" target="_blank">Contacts Directory</a>.&nbsp;</p>\n'
else: 
    def muni_c_lst(info):
        muni_contact = []
        address = info.df.Address[0]
        city = info.df.City[0]
        zipcode = info.df.Zip[0]
        phone1 = info.df.Phone[0]
        phone = str(phone1[0:3] + '-' + phone1[3:6] + '-' + phone1[6:11])
        fax1 = info.df.Fax[0]
        fax = str(fax1[0:3] + '-' + fax1[3:6] + '-' + fax1[6:11])
        website = info.df.Website[0]
        email = info.df.Email[0]
        print(address, city, zipcode, phone, fax, website, email)
        muni_contact.append("<p style='text-align:left'><u><b>Municipality Contact Information:</b></u></p>")
        muni_contact.append("<p style='text-align:center'><b>" + str(info.df.EntityName[0]) + '</b></p>')
        muni_contact.append("<p style='text-align:center'>" + str(address) + ", " + str(city) + ", AK" + " " + str(zipcode) + '<p>&nbsp;</p>\n\n')
        muni_contact.append("<p style='text-align:center'>Phone Number: " + str(phone) + '</p>')
        muni_contact.append("<p style='text-align:center'>Fax Number: " + str(fax) + '</p><p>&nbsp;</p>\n\n')
        muni_contact.append("<p style='text-align:center'>Website: " + str(website) + '</p>')
        muni_contact.append("<p style='text-align:center'>" + str(email) + '</p><p>&nbsp;</p>\n\n')
        return(' '.join(muni_contact))

    muni_c_lst(contact_info)
    panel_contacts["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<div>'+ str(muni_c_lst(contact_info)) + '</div>\n\n<p>&nbsp;</p>\n\n<p>To find all community contacts and to download mailing labels visit the <a href="http://dcced.maps.arcgis.com/apps/webappviewer/index.html?id=148ae1328f9549acb46cfa80366ed37e/data" target="_blank">Contacts Directory</a>.&nbsp;</p>\n'
    clonedStorymap.save()

PO Box 281 Nome 99762 907-443-6663 907-443-5345 http://www.nomealaska.org/ bhammond@nomealaska.org
PO Box 281 Nome 99762 907-443-6663 907-443-5345 http://www.nomealaska.org/ bhammond@nomealaska.org


In [26]:
### CONTACTS PANEL ####

tribe_lyr = gis.content.get("c37a47a8196f409aa6cdcf2a2df097bc")
tribe_contact_info = tribe_lyr.layers[0].query(where="City = '" + str(community) + "'", out_fields='EntityName, Address, City, Zip, Phone, Fax, Website, Email')
tribe_output = str(tribe_contact_info.df)
if tribe_output ==  'Empty DataFrame\nColumns: []\nIndex: []':
    print(str(community) + ' is not served by a federally recognized tribe. No FRT data will be written to panel.')
else:
    def tribe_info(info):
        tribe_contact = []
        tribe = info.df.EntityName[0]
        tribe_address = info.df.Address[0]
        tribe_city = info.df.City[0]
        tribe_zip = info.df.Zip[0]
        tribe_phone = info.df.Phone[0]
        tribe_fax = info.df.Fax[0]
        tribe_web = info.df.Website[0]
        tribe_email = info.df.Email[0]
        print(tribe, tribe_address, tribe_city, tribe_zip, tribe_phone, tribe_fax, tribe_web, tribe_email)
        tribe_contact.append("<p style='text-align:left'><u><b>Federally Recognized Tribe Contact Information:</b></u></p>")
        tribe_contact.append("<p style='text-align:center'><b>" + str(info.df.EntityName[0]) + '</b></p>')
        tribe_contact.append("<p style='text-align:center'>" + str(tribe_address) + ", " + str(tribe_city) + ", AK" + " " + str(tribe_zip) + '<p>&nbsp;</p>\n\n')
        tribe_contact.append("<p style='text-align:center'>Phone Number: " + str(tribe_phone) + '</p>')
        tribe_contact.append("<p style='text-align:center'>Fax Number: " + str(tribe_fax) + '</p><p>&nbsp;</p>\n\n')
        tribe_contact.append("<p style='text-align:center'>Website: " + str(tribe_web) + '</p>')
        tribe_contact.append("<p style='text-align:center'>" + str(tribe_email) + '</p><p>&nbsp;</p>\n\n')
        return(' '.join(tribe_contact))
    embed_tribe = tribe_info(tribe_contact_info)
    panel_contacts["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<div>'+ str(muni_c_lst(contact_info)) + '</div>\n\n<p>&nbsp;</p>\n\n<div>' + str(embed_tribe) + '</div><p>To find all community contacts and to download mailing labels visit the <a href="http://dcced.maps.arcgis.com/apps/webappviewer/index.html?id=148ae1328f9549acb46cfa80366ed37e/data" target="_blank">Contacts Directory</a>.&nbsp;</p>\n'
    clonedStorymap.save()

Native Village of Council PO Box 2050 Nome 99762 (907) 443-7649 (907) 443-5965 http://www.kawerak.org tc.cou@kawerak.org
PO Box 281 Nome 99762 907-443-6663 907-443-5345 http://www.nomealaska.org/ bhammond@nomealaska.org


In [27]:
### CONTACTS PANEL ###

corp_lyr = gis.content.get("2c5868dcf56e4631a155c1a8985c9e26")
corp_contact_info = corp_lyr.layers[0].query(where="CommunityName = '" + str(community) +"'", out_fields = 'EntityName, Address, City, State, Zip, Phone, Fax, Website, Email')
corp_output = str(corp_contact_info.df)
if corp_output ==  'Empty DataFrame\nColumns: []\nIndex: []':
    print(str(community) + ' is not served by a federally recognized tribe. No Village Regional Corporation data will be written to panel.')
else:
    def corp_info(info):
        corp_contact = []
        corp = info.df.EntityName[0]
        corp_address = info.df.Address[0]
        corp_city = info.df.City[0]
        corp_zip = info.df.Zip[0]
        corp_phone = info.df.Phone[0]
        corp_fax = info.df.Fax[0]
        corp_web = info.df.Website[0]
        corp_email = info.df.Email[0]
        print(corp, corp_address, corp_city, corp_zip, corp_phone, corp_fax, corp_web, corp_email)
        corp_contact.append("<p style='text-align:left'><u><b>Native Village Corporation Contact Information:</b></u></p>")
        corp_contact.append("<p style='text-align:center'><b>" + str(info.df.EntityName[0]) + '</b></p>')
        corp_contact.append("<p style='text-align:center'>" + str(corp_address) + ", " + str(corp_city) + ", AK" + " " + str(corp_zip) + '<p>&nbsp;</p>\n\n')
        corp_contact.append("<p style='text-align:center'>Phone Number: " + str(corp_phone) + '</p>')
        corp_contact.append("<p style='text-align:center'>Fax Number: " + str(corp_fax) + '</p><p>&nbsp;</p>\n\n')
        corp_contact.append("<p style='text-align:center'>Website: " + str(corp_web) + '</p>')
        corp_contact.append("<p style='text-align:center'>Email: " + str(corp_email) + '</p><p>&nbsp;</p>\n\n')
        return(' '.join(corp_contact))
    embed_corp = corp_info(corp_contact_info)
    panel_contacts["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<div>'+ str(muni_c_lst(contact_info)) + '</div>\n\n<p>&nbsp;</p>\n\n<div>' + str(embed_tribe) + '</div><p>\n\n<p>&nbsp;</p>\n\n<div>' + str(embed_corp) + '</div><p>\n\n<p>&nbsp;</p>To find all community contacts and to download mailing labels visit the <a href="http://dcced.maps.arcgis.com/apps/webappviewer/index.html?id=148ae1328f9549acb46cfa80366ed37e/data" target="_blank">Contacts Directory</a>.&nbsp;</p>\n'
    clonedStorymap.save()

Sitnasuak Native Corporation P.O. Box 905 Nome 99762 9073871200 9074433063 http://www.snc.org None
PO Box 281 Nome 99762 907-443-6663 907-443-5345 http://www.nomealaska.org/ bhammond@nomealaska.org
